### stage_model_regressor

prediction models of chg_of_rank_in_stage on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.linear_model.ridge import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVR
from sklearn.utils import shuffle
from sklearn import metrics
import xgboost as xgb

In [3]:
# bulid regression model
regressors = ['currank','avgrank','dice','lasso','ridge','rf','svr','xgb']
def get_regressor(regressor = 'lr'):
    if regressor == "lasso":
        clf = LassoCV(cv=5, random_state=0)
    elif regressor == "ridge":
        clf = RidgeCV(alphas=np.logspace(-6, 6, 13))
    elif regressor == "rf":
        clf = RandomForestRegressor(n_estimators=100)
    elif regressor == 'svr':
        clf = SVR(kernel='rbf')
    elif regressor == 'xgb':
        clf = xgb.XGBRegressor(objective="reg:linear", random_state=42, max_depth=3)
    elif regressor == 'dice':
        clf = RandomDice('1234')
    elif regressor == 'currank':
        clf = CurRank()
    elif regressor == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        #pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        pred_y_avg = pred_y
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    mae = metrics.mean_absolute_error(test_y, pred_y) 
    rmse = math.sqrt(metrics.mean_squared_error(test_y, pred_y))
    r2 = metrics.r2_score(test_y, pred_y)
    print('rmse=%.2f, mae=%.2f, r2=%.2f'%(rmse, mae, r2))
    return rmse, mae, r2
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,2:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,2:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    return train, test, train_x, train_y, test_x, test_y


def regressor_model(name='svr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_regressor(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [4]:
#load data
suffix='-withneighbor-newfeatures'
stagedata = pd.read_csv('stage-2018%s.csv'%suffix)
stagedata.fillna(0, inplace=True)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 35 columns):
Unnamed: 0                   805 non-null int64
target                       805 non-null int64
eventid                      805 non-null int64
car_number                   805 non-null int64
stageid                      805 non-null int64
firststage                   805 non-null int64
pit_in_caution               805 non-null int64
start_position               805 non-null int64
start_rank                   805 non-null int64
start_rank_ratio             805 non-null float64
top_pack                     805 non-null int64
bottom_pack                  805 non-null int64
average_rank                 805 non-null float64
average_rank_all             805 non-null float64
change_in_rank               805 non-null int64
change_in_rank_all           805 non-null float64
rate_of_change               805 non-null int64
rate_of_change_all           805 non-null float64
laptime_green_mean_pr

### model on data split by event

In [5]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(regressors)
print('cols:%s'%cols)
retdf0 = pd.DataFrame([],columns=cols)
retdf1 = pd.DataFrame([],columns=cols)

eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec0 = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    rec1 = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc0 = [0 for x in range(len(regressors))]
    acc1 = [0 for x in range(len(regressors))]
    for idx, clf in enumerate(regressors):
        acc0[idx] = regressor_model(clf)[0]
        acc1[idx] = regressor_model(clf)[2]

    rec0.extend(acc0)
    rec1.extend(acc1)
    #print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec0],columns=cols)
    retdf0 = pd.concat([retdf0, df])        
    
    df = pd.DataFrame([rec1],columns=cols)
    retdf1 = pd.concat([retdf1, df])        

    
retdf0.to_csv('regressors_stagedata_splitbyevent%s_rmse.csv'%suffix)
retdf1.to_csv('regressors_stagedata_splitbyevent%s_r2.csv'%suffix)

df_event_rmse = retdf0
df_event_r2 = retdf1

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lasso', 'ridge', 'rf', 'svr', 'xgb']
Testset = Phoenix
[*] predict with currank model
rmse=4.73, mae=3.20, r2=-0.00
[*] predict with currank model
rmse=4.73, mae=3.20, r2=-0.00
[*] predict with avgrank model
rmse=5.61, mae=3.83, r2=-0.40
[*] predict with avgrank model
rmse=5.61, mae=3.83, r2=-0.40
[*] predict with dice model
rmse=7.01, mae=4.88, r2=-1.19
[*] predict with dice model
rmse=7.01, mae=4.88, r2=-1.19
[*] predict with lasso model
rmse=4.40, mae=2.95, r2=0.14
[*] predict with lasso model
rmse=4.40, mae=2.95, r2=0.14
[*] predict with ridge model
rmse=4.40, mae=2.91, r2=0.13
[*] predict with ridge model
rmse=4.40, mae=2.91, r2=0.13
[*] predict with rf model
rmse=4.45, mae=3.19, r2=0.11
[*] predict with rf model
rmse=4.46, mae=3.18, r2=0.11
[*] predict with svr model
rmse=4.73, mae=3.16, r2=0.00
[*] predict with svr model
rmse=4.73, mae=3.16, r2=0.00
[*] predict with xgb model
[15:36:37] WA

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


rmse=4.14, mae=2.89, r2=0.24
[*] predict with xgb model
[15:36:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=4.14, mae=2.89, r2=0.24
Testset = Indy500
[*] predict with currank model
rmse=6.17, mae=3.90, r2=-0.00
[*] predict with currank model
rmse=6.17, mae=3.90, r2=-0.00
[*] predict with avgrank model
rmse=6.94, mae=4.75, r2=-0.27
[*] predict with avgrank model
rmse=6.94, mae=4.75, r2=-0.27
[*] predict with dice model
rmse=7.55, mae=5.28, r2=-0.50
[*] predict with dice model
rmse=7.55, mae=5.28, r2=-0.50
[*] predict with lasso model
rmse=5.75, mae=4.05, r2=0.13
[*] predict with lasso model
rmse=5.75, mae=4.05, r2=0.13
[*] predict with ridge model
rmse=5.66, mae=3.97, r2=0.15
[*] predict with ridge model
rmse=5.66, mae=3.97, r2=0.15
[*] predict with rf model
rmse=5.68, mae=4.33, r2=0.15
[*] predict with rf model
rmse=5.80, mae=4.50, r2=0.11
[*] predi

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


rmse=5.73, mae=4.23, r2=0.13
[*] predict with xgb model
[15:36:39] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=5.73, mae=4.23, r2=0.13
Testset = Texas
[*] predict with currank model
rmse=4.26, mae=2.60, r2=-0.01
[*] predict with currank model
rmse=4.26, mae=2.60, r2=-0.01
[*] predict with avgrank model
rmse=5.48, mae=3.73, r2=-0.67
[*] predict with avgrank model
rmse=5.48, mae=3.73, r2=-0.67
[*] predict with dice model
rmse=6.73, mae=4.68, r2=-1.52
[*] predict with dice model
rmse=6.73, mae=4.68, r2=-1.52
[*] predict with lasso model
rmse=3.91, mae=2.60, r2=0.15
[*] predict with lasso model
rmse=3.91, mae=2.60, r2=0.15
[*] predict with ridge model
rmse=3.64, mae=2.38, r2=0.26
[*] predict with ridge model
rmse=3.64, mae=2.38, r2=0.26
[*] predict with rf model
rmse=3.95, mae=2.67, r2=0.13
[*] predict with rf model
rmse=4.00, mae=2.73, r2=0.11
[*] predict

...

rmse=3.26, mae=2.68, r2=-0.80
[*] predict with xgb model
[15:36:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=3.26, mae=2.68, r2=-0.80
Testset = Gateway
[*] predict with currank model
rmse=3.41, mae=2.16, r2=-0.00
[*] predict with currank model
rmse=3.41, mae=2.16, r2=-0.00
[*] predict with avgrank model
rmse=4.30, mae=2.81, r2=-0.59
[*] predict with avgrank model
rmse=4.30, mae=2.81, r2=-0.59
[*] predict with dice model
rmse=5.89, mae=4.16, r2=-1.98
[*] predict with dice model
rmse=5.89, mae=4.16, r2=-1.98
[*] predict with lasso model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/coo

rmse=3.23, mae=2.32, r2=0.10
[*] predict with lasso model
rmse=3.23, mae=2.32, r2=0.10
[*] predict with ridge model
rmse=2.77, mae=2.02, r2=0.34
[*] predict with ridge model
rmse=2.77, mae=2.02, r2=0.34
[*] predict with rf model
rmse=3.27, mae=2.38, r2=0.08
[*] predict with rf model
rmse=3.30, mae=2.40, r2=0.06
[*] predict with svr model
rmse=3.35, mae=2.21, r2=0.04
[*] predict with svr model
rmse=3.35, mae=2.21, r2=0.04
[*] predict with xgb model
[15:36:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


rmse=2.95, mae=2.18, r2=0.25
[*] predict with xgb model
[15:36:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=2.95, mae=2.18, r2=0.25


### model on data split by stage

In [6]:
retdf0 = pd.DataFrame([],columns=cols)
retdf1 = pd.DataFrame([],columns=cols)

for stageid in range(8):
    train, test, train_x, train_y, test_x, test_y =split_by_stageid(stagedata, stageid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec0 = ['stage%d'%stageid,train_x.shape[0],test_x.shape[0],test_distribution]
    rec1 = ['stage%d'%stageid,train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc0 = [0 for x in range(len(regressors))]
    acc1 = [0 for x in range(len(regressors))]
    for idx, clf in enumerate(regressors):
        acc0[idx] = regressor_model(clf)[0]
        acc1[idx] = regressor_model(clf)[2]

    rec0.extend(acc0)
    rec1.extend(acc1)
    #print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec0],columns=cols)
    retdf0 = pd.concat([retdf0, df])  
    
    df = pd.DataFrame([rec1],columns=cols)
    retdf1 = pd.concat([retdf1, df])  

retdf0.to_csv('regressor_stagedata_splitbystage%s_rmse.csv'%suffix)
retdf1.to_csv('regressor_stagedata_splitbystage%s_r2.csv'%suffix)

df_stage_rmse = retdf0
df_stage_r2 = retdf1

[*] predict with currank model
rmse=4.75, mae=2.85, r2=-0.00
[*] predict with currank model
rmse=4.75, mae=2.85, r2=-0.00
[*] predict with avgrank model
rmse=5.87, mae=3.91, r2=-0.53
[*] predict with avgrank model
rmse=5.87, mae=3.91, r2=-0.53
[*] predict with dice model
rmse=6.16, mae=4.36, r2=-0.68
[*] predict with dice model
rmse=6.16, mae=4.36, r2=-0.68
[*] predict with lasso model
rmse=4.85, mae=3.19, r2=-0.05
[*] predict with lasso model
rmse=4.85, mae=3.19, r2=-0.05
[*] predict with ridge model
rmse=4.96, mae=3.25, r2=-0.09
[*] predict with ridge model
rmse=4.96, mae=3.25, r2=-0.09
[*] predict with rf model
rmse=5.02, mae=3.39, r2=-0.12
[*] predict with rf model
rmse=5.00, mae=3.34, r2=-0.11
[*] predict with svr model
rmse=4.78, mae=2.97, r2=-0.01
[*] predict with svr model
rmse=4.78, mae=2.97, r2=-0.01
[*] predict with xgb model
[15:36:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprec

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


rmse=28.76, mae=7.75, r2=-36.74
[*] predict with lasso model
rmse=28.76, mae=7.75, r2=-36.74
[*] predict with ridge model
rmse=34.26, mae=15.09, r2=-52.58
[*] predict with ridge model
rmse=34.26, mae=15.09, r2=-52.58
[*] predict with rf model
rmse=5.02, mae=4.03, r2=-0.15
[*] predict with rf model
rmse=4.94, mae=3.94, r2=-0.11
[*] predict with svr model
rmse=4.77, mae=2.91, r2=-0.04
[*] predict with svr model
rmse=4.77, mae=2.91, r2=-0.04
[*] predict with xgb model
[15:36:51] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=5.36, mae=4.06, r2=-0.31
[*] predict with xgb model
[15:36:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=5.36, mae=4.06, r2=-0.31
[*] predict with currank model
rmse=4.90, mae=2.74, r2=-0.01
[*] predic

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predic

rmse=4.80, mae=2.96, r2=0.04
[*] predict with lasso model
rmse=4.80, mae=2.96, r2=0.04
[*] predict with ridge model
rmse=5.18, mae=3.36, r2=-0.12
[*] predict with ridge model
rmse=5.18, mae=3.36, r2=-0.12
[*] predict with rf model


...

rmse=5.04, mae=3.15, r2=-0.00
[*] predict with xgb model
[15:37:00] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=5.04, mae=3.15, r2=-0.00
[*] predict with currank model
rmse=3.85, mae=2.26, r2=-0.00
[*] predict with currank model
rmse=3.85, mae=2.26, r2=-0.00
[*] predict with avgrank model
rmse=4.36, mae=2.79, r2=-0.28
[*] predict with avgrank model
rmse=4.36, mae=2.79, r2=-0.28
[*] predict with dice model
rmse=7.17, mae=4.53, r2=-2.47
[*] predict with dice model
rmse=7.17, mae=4.53, r2=-2.47
[*] predict with lasso model
rmse=3.62, mae=2.58, r2=0.12
[*] predict with lasso model
rmse=3.62, mae=2.58, r2=0.12
[*] predict with ridge model
rmse=3.31, mae=2.41, r2=0.26
[*] predict with ridge model
rmse=3.31, mae=2.41, r2=0.26
[*] predict with rf model
rmse=3.23, mae=2.47, r2=0.30
[*] predict with rf model
rmse=3.13, mae=2.38, r2=0.34
[*] predict with svr mode

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


rmse=3.64, mae=2.78, r2=0.11
[*] predict with xgb model
[15:37:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=3.64, mae=2.78, r2=0.11
[*] predict with currank model
rmse=2.68, mae=1.81, r2=-0.01
[*] predict with currank model
rmse=2.68, mae=1.81, r2=-0.01
[*] predict with avgrank model
rmse=3.05, mae=2.28, r2=-0.31
[*] predict with avgrank model
rmse=3.05, mae=2.28, r2=-0.31
[*] predict with dice model
rmse=5.99, mae=4.19, r2=-4.07
[*] predict with dice model
rmse=5.99, mae=4.19, r2=-4.07
[*] predict with lasso model
rmse=2.20, mae=1.62, r2=0.32
[*] predict with lasso model
rmse=2.20, mae=1.62, r2=0.32
[*] predict with ridge model
rmse=2.62, mae=2.01, r2=0.03
[*] predict with ridge model
rmse=2.62, mae=2.01, r2=0.03
[*] predict with rf model
rmse=2.97, mae=2.44, r2=-0.24
[*] predict with rf model
rmse=2.75, mae=2.24, r2=-0.06
[*] predict with svr mode

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


rmse=3.28, mae=2.62, r2=-0.51
[*] predict with xgb model
[15:37:06] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse=3.28, mae=2.62, r2=-0.51


In [7]:
df_event_rmse

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lasso,ridge,rf,svr,xgb
0,Phoenix,691,114,"+:38,0:16,-:60",4.734161,5.606406,7.008766,4.400094,4.402753,4.454113,4.729814,4.135296
0,Indy500,580,225,"+:82,0:47,-:96",6.165135,6.936325,7.545124,5.750060,5.662937,5.680989,6.156733,5.733629
0,Texas,678,127,"+:39,0:34,-:54",4.257462,5.478100,6.726373,3.910683,3.643120,3.954702,4.186327,3.973756
0,Iowa,696,109,"+:42,0:28,-:39",3.981609,5.800698,6.453077,3.719822,3.976597,4.126235,3.901714,4.428711
0,Pocono,679,126,"+:29,0:61,-:36",2.475275,2.928532,5.611072,100.878775,160.688382,3.462487,2.431986,3.259853
0,Gateway,701,104,"+:34,0:28,-:42",3.412364,4.301330,5.885935,3.228556,2.774166,3.274902,3.348218,2.954373


In [8]:
df_event_r2

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lasso,ridge,rf,svr,xgb
0,Phoenix,691,114,"+:38,0:16,-:60",-0.000416,-0.403019,-1.192692,0.135792,0.134747,0.110878,0.001421,0.236678
0,Indy500,580,225,"+:82,0:47,-:96",-0.003011,-0.269636,-0.502288,0.127500,0.153740,0.112185,-0.000279,0.132480
0,Texas,678,127,"+:39,0:34,-:54",-0.007943,-0.668763,-1.515918,0.149567,0.261957,0.108269,0.025458,0.121914
0,Iowa,696,109,"+:42,0:28,-:39",0.000000,-1.122479,-1.626736,0.127175,0.002516,-0.067230,0.039729,-0.237193
0,Pocono,679,126,"+:29,0:61,-:36",-0.038432,-0.453555,-4.336088,-1723.768826,-4375.231135,-1.539879,-0.002429,-0.801057
0,Gateway,701,104,"+:34,0:28,-:42",-0.001344,-0.591028,-1.979225,0.103626,0.338184,0.062069,0.035950,0.249410


In [9]:
df_stage_rmse

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lasso,ridge,rf,svr,xgb
0,stage0,153,652,"+:221,0:167,-:264",4.747780,5.870429,6.158315,4.852986,4.960882,5.021723,4.777338,5.130541
0,stage1,288,517,"+:186,0:136,-:195",4.682987,5.526302,6.856642,28.756027,34.262713,5.017130,4.770325,5.355350
0,stage2,421,384,"+:140,0:112,-:132",4.903761,5.681358,7.051706,4.795480,5.182141,4.646937,4.988875,4.769602
0,stage3,547,258,"+:91,0:89,-:78",4.734420,5.541856,7.011065,4.776015,4.729745,4.631724,4.822095,4.995320
0,stage4,657,148,"+:48,0:53,-:47",4.914815,5.646990,6.450435,4.866051,4.882638,4.611071,4.951842,4.991917
0,stage5,725,80,"+:26,0:29,-:25",5.054701,5.848562,6.650188,4.752693,4.593081,4.320071,5.078110,5.044269
0,stage6,767,38,"+:11,0:13,-:14",3.852545,4.358174,7.174516,3.621287,3.309558,3.231518,3.852935,3.636465
0,stage7,789,16,"+:4,0:4,-:8",2.680951,3.045832,5.994789,2.203170,2.622328,2.968728,2.664454,3.276612


In [10]:
df_stage_r2

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lasso,ridge,rf,svr,xgb
0,stage0,153,652,"+:221,0:167,-:264",-0.000316,-0.529311,-0.682983,-0.045139,-0.092129,-0.109783,-0.012810,-0.168106
0,stage1,288,517,"+:186,0:136,-:195",-0.000986,-0.393963,-1.145879,-36.743311,-52.582843,-0.113881,-0.038672,-0.309055
0,stage2,421,384,"+:140,0:112,-:132",-0.005719,-0.349964,-1.079728,0.038205,-0.123147,0.109552,-0.040934,0.048558
0,stage3,547,258,"+:91,0:89,-:78",-0.019516,-0.396919,-1.235778,-0.037509,-0.017504,0.070794,-0.057626,-0.134978
0,stage4,657,148,"+:48,0:53,-:47",-0.008557,-0.331435,-0.737257,0.011357,0.004606,0.144512,-0.023810,-0.040449
0,stage5,725,80,"+:26,0:29,-:25",-0.004151,-0.344332,-0.738105,0.112256,0.170882,0.189215,-0.013473,-0.000011
0,stage6,767,38,"+:11,0:13,-:14",-0.000747,-0.280672,-2.470676,0.115791,0.261469,0.339544,-0.000949,0.108364
0,stage7,789,16,"+:4,0:4,-:8",-0.013774,-0.308505,-4.068871,0.315364,0.030077,-0.064932,-0.001336,-0.514305
